## ESA 2024 Special Session - SS 08

# Accessing Data from NASA’s First Biodiversity-Focused Airborne and Field Campaign, BioSCape
**Monday, August 5, 2024, 11:45 - 1:15 PM PDT**
- Organizer - Erin Hestir, University of California Merced
- Co-Organizers - In Person: Anabelle Cardoso, Philip Brodrick, Michele Thornton, Erin Hestir
- Co-Organizers - Additional: Rupesh Shrestha, Glenn Moncrieff, Adam Wilson

## Session Overview

<center><img src="NotebookImages/121229-87.png"/></center>

**ESA 2024 Special Session: SS 08 - Accessing Data from NASA’s First Biodiversity-Focused Airborne and Field Campaign, BioSCape**

[BioSCape](https://www.bioscape.io/), the Biodiversity Survey of the Cape, is NASA’s first biodiversity-focused airborne and field campaign that was conducted in South Africa in 2023. BioSCape’s primary objective is to study the structure, function, and composition of the region’s ecosystems, and how and why they are changing. 

BioSCape's airborne dataset is unprecedented, with AVIRIS-NG, PRISM, and HyTES imaging spectrometers capturing spectral data across the UV, visible and infrared at high resolution and LVIS acquiring coincident full-waveform lidar. BioSCape's field dataset is equally impressive, with 18 PI-led projects collecting data ranging from the diversity and phylogeny of plants, kelp and phytoplankton, eDNA, landscape acoustics, plant traits, blue carbon accounting, and more

BioSCape is committed to Open Science. All our datasets will be delivered to one of NASA's Distributed Archive and Analysis Centers. In this ESA special session we will: 

1) Give an overview of BioSCape’s research activities, 
2) Discuss the ways in which BioSCape data has been optimized for easier uptake by new users, and
3) Demonstrate where and how one can access BioSCape’s field and airborne data sets.

The goal of this session is to encourage wider awareness and use of BioSCape data and promote its application in research and biodiversity conservation.

----
# Tutorial: Exploring BioSCape AVIRIS-NG L2A Reflectance Data
----

### BioSCape AVIRIS-Next Generation Data

Airborne Visible InfraRed Imaging Spectrometer - Next Generation (AVIRIS-NG) provides high signal-to-noise ratio imaging spectroscopy measurements in the solar reflected spectral range. AVIRIS-NG measures the wavelength range from 380 nm to 2510 nm with 5 nm sampling.  More information about the AVIRIS-NG instrument is [here](https://avirisng.jpl.nasa.gov/aviris-ng.html).

Preliminary BioSCape AVIRIS-NG Data available here:  https://popo.jpl.nasa.gov/avng/y23_bioscape/

BioSCape Airborne data interactive viewer: **BioSCape Data Portal**: https://popo.jpl.nasa.gov/mmgis-aviris/?mission=BIOSCAPE 

- Data are orthorectified
- Flight lines are provided as smaller, more manageable sections of data we'll refer to as "scenes"
- Within a flight line, adjacent scenes are seamless
- To date, data are in binary/header (ENVI) formats
- Data are calibrated to at-sensor radiance
- `L2a Data`: Surface Reflectance and Uncertainty are available

### Learning Objectives
### In this tutorial, we'll use python methods to: 

1. Use GDAL methods to open and explore AVIRIS-NG data
2. Create virtual raster (vrt) to stitch adjacent scences
3. Transform a lat/lon location to row colume locations
4. Plot a location's spectral profile

----

## AVIRIS-NG Data Files
### **From the [BioSCape Data Portal](https://popo.jpl.nasa.gov/mmgis-aviris/?mission=BIOSCAPE), using the coordinates of the Brackenburn Reserve, we can determine and download scenes of interest**
- Brackenburn Priviate Nature Reserve Coordinates:  **`-33.978573`**, **`23.474745`**

##### Corresponding L2 Files
- ang20231110t081307_006_L2A_OE_main_27577724_RFL_ORT_QL.tif
- ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT_QL.tif
- ang20231110t081307_006_L2A_OE_main_27577724_RFL_ORT.tar.gz
- ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT.tar.gz


| Dataset | Description | 
| -------- | --- |
| *_RFL_ORT_QL.tif | Reflectance Quick Look Image (3 band) |
| *-RFL_ORT | Reflectance ENVI binary file (425 band)|
| *_RFL_ORT.hdr | Reflectance ENVI header file (txt file)|

<center><img src="NotebookImages/ANG_naming.jpg"/></center>


## Study Area
### For this tutorial, the [Brackenburn Private Nature Reserve](https://plcnetwork.co.za/member/121/Brackenburn-Private-Nature-Reserve/) was selected as a study area of interest.

<center><img src="NotebookImages/brackenburn.PNG"/></center>

### Import python modules

- **`gdal`** a popular Geospatial Data Abstraction Library.  A translator library for raster and vector geospatial data formats.
- **`NumPy`** is the fundamental package for scientific computing within Python and provides an N-dimensional array object suitable for multidimensional files.

In [ ]:
# python imports 
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from spectral.io import envi
from pyproj import Proj
import os
from glob import glob

## Examine Adjacent Quick Look geoTIFF Scenes using GDAL

### GDAL Raster Dataset

- **`GDAL`** (Geospatial Data Abstraction Library) is a translator library for raster and vector geospatial data formats
- We've already downloaded 2 adjacent scenes of AVIRIS-NG data (Quicklook and L2A Reflectance) in the Brackenburg Private Nature Reserve

In [ ]:
for geotiff in glob("data/*.tif"):
    print(f"{os.path.basename(geotiff)}")

### GDAL to get the number of `bands`, `rows`, and `columns` in the Quicklook files

In [ ]:
# open and read tif files for comparison

ql_006 = gdal.Open('data/ang20231110t081307_006_L2A_OE_main_27577724_RFL_ORT_QL.tif')
ql_007 = gdal.Open('data/ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT_QL.tif')

nbands_006 = ql_006.RasterCount
ncols_006 = ql_006.RasterXSize
nrows_006 = ql_006.RasterYSize

nbands_007 = ql_007.RasterCount
ncols_007 = ql_007.RasterXSize
nrows_007 = ql_007.RasterYSize

print(f"Bands_006:\t{nbands_006}")
print(f"Rows_006:\t{nrows_006}")
print(f"Cols_006:\t{ncols_006}")
print("\n")
print(f"Bands_007:\t{nbands_007}")
print(f"Rows_007:\t{nrows_007}")
print(f"Cols_007:\t{ncols_007}")

In [ ]:
ql_007.GetProjection()

### Create a Virtual Raster of the 2 Adjacent AVIRIS-NG Scenes

- A **virtual raster** (VRT) is a GDAL method of combining multiple raster tiles into one file. It enables visualisation of whole raster datasets and faster navigation around your dataset in GIS software.
- Using 2 adjacent flight line AVIRIS-NG quicklook images (3-band true color geoTIFF file) we'll use GDAL to create a virtual raster (vrt). 

In [ ]:
#build vrt - using AVIRIS-NG quicklook images 

# running gdalbuilvrt as a gdal command line using the '!' in front says do this as a command, not as python
# using scences from the Bracnenburn Private Nature Reserve

!gdalbuildvrt brack_mosaic.vrt data/*.tif

# read in vrt, ql ~ quicklook
brack_vrt = gdal.Open('brack_mosaic.vrt')

nbands_vrt = brack_vrt.RasterCount
ncols_vrt = brack_vrt.RasterXSize
nrows_vrt = brack_vrt.RasterYSize

print(f"Bands_vrt:\t{nbands_vrt}")
print(f"Rows_vrt:\t{nrows_vrt}")
print(f"Cols_vrt:\t{ncols_vrt}")


## Examine AVIRIS-NG Quiklook Reflectance Data as a Numerical Array

GDAL is a powerful library when it comes to accessing geospatial raster data, but it does not provide many functionalities for doing calculations. For more advanced computing, we will **read in the raster data** as a numerical array in order to use the capabilities in the NumPy library.

You can convert an existing Gdal Dataset or a Band into a **numpy array** with the **`ReadAsArray()`** function.

In [ ]:
# Recall that we created the quicklook virtual raster (ql_vrt) from two adjacent scenes
brack_array = brack_vrt.ReadAsArray()

# look at the shape. GDAL reads the array as band,x,y.  
print(brack_array.shape)                     

In [ ]:
#To plot the array with matplotlib, the data need to be arranged as y, x, band.
# We'll transpose the data
trans_array = brack_array.transpose((1,2,0))
print(trans_array.shape)

In [ ]:
# To demonstrate that data needs transposed, this will give an error because the data needs to be transposed
# This is the error:  TypeError: Invalid shape (3, 1268, 1216) for image data
#plt.imshow(brack_array)

### The imshow() function in the pyplot module of matplotlib library is used to display data as an image. 

In [ ]:
plt.rcParams['figure.figsize'] = [10,7]
plt.imshow(trans_array)

----

##  AVIRIS-NG L2A Reflectance DataCube - Image Spectroscopy (ENVI format)

#### Recall that we examined 2 adjacent tiles from the same flight path for the following files
- Recall we downloaded files 
- ang20231110t081307_006_L2A_OE_main_27577724_RFL_ORT_QL.tif
- ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT_QL.tif

##### Corresponding Reflectance Files
- ang20231110t081307_006_L2A_OE_main_27577724_RFL_ORT.tar.gz
- ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT.tar.gz

In [ ]:
# Need to untar/uncompress data files
#!tar -zxvf aang20231110t081307_006_L2A_OE_main_27577724_RFL_ORT.tar.gz
#!tar -zxvf ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT.tar.gz

In [ ]:
for files in glob("data/*"):
    print(f"{os.path.basename(files)}")

----
### The current AVIRIS-NG Reflectance Files are in ENVI file formats which are binary/header file pairs

### Let's examine the header file of one of the ENVI scenes that we downloaded:

In [ ]:
hdr_f = 'data/ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT.hdr'
with open(hdr_f, mode='r') as hdr:
    lines = (hdr.read())
    print(lines)

### Open the ENVI Reflectance File as a GDAL raster dataset and print the dimensions:
#### GDAL to get the number of bands, rows, and columns in the Reflectance file

In [ ]:
# Open the ENVI file and read the file bands, row, cols

rfl_007_open = gdal.Open('data/ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT')

nbands = rfl_007_open.RasterCount
nrows = rfl_007_open.RasterYSize
ncols = rfl_007_open.RasterXSize

print(f"Bands:\t{nbands}")
print(f"Rows:\t{nrows}")
print(f"Cols:\t{ncols}")

#### GDAL to get Metadata and Projection Information
- The GDAL **GetMetadata** function can read some metadata from images and organizes the metadata in the form of a dictionary
- The GDAL **GetProjection** function to obtain file projection information

In [ ]:
rfl_007_open.GetMetadata()

In [ ]:
print("ENVI image WKT: \n"+str(rfl_007_open.GetProjection()))

### UTM zone 34S, datum WGS-84 = EPSG Code: 32734 

#### Read one spectral band array and plot it
- Red:  Band 57 is ~657nm (the center of the Landsat red band)
- Green: Band 38 is ~562nm (the center of the Landsat green band)
- Blue:  Band 22 is ~482nm (the center of the Landsat blue band)

In [ ]:
#### uncomment one of the GetRaseterBand lines to run
img_red = rfl_007_open.GetRasterBand(57).ReadAsArray()  # Band 57 is 657nm (the center of the Landsat red band)
#img_green = rfl_007_open.GetRasterBand(38).ReadAsArray()  # Band 38 is 562nm (the center of the Landsat green band)
#img_blue = rfl_007_open.GetRasterBand(22).ReadAsArray()  # Band 22 is 482nm (the center of the Landsat blue band)
plt.rcParams['figure.figsize'] = [7,17]
plt.rcParams['figure.dpi'] = 100
plt.imshow(img_red, vmin=0, vmax=.20)
#plt.imshow(img_green, vmin=0, vmax=0.20)
#plt.imshow(img_blue, vmin=0, vmax=0.20)
plt.colorbar(fraction=0.04, pad=0.04)
plt.show()

## Create a Red, Green, Blue Composite and Visualize

In [ ]:
# get r,g,b arrays
red, green, blue = rfl_007_open.GetRasterBand(57).ReadAsArray(), rfl_007_open.GetRasterBand(38).ReadAsArray(), rfl_007_open.GetRasterBand(22).ReadAsArray()

# set fill values (-9999.) to 0 for each array
red[red == -9999.], green[green == -9999.], blue[blue == -9999.] = 0, 0, 0

# function scales reflectance values to 8 bits
scale8bit = lambda a: ((a - a.min()) * (1/(a.max() - a.min()) * 255)).astype('uint8')

# get 8bit arrays for each band
red8, green8, blue8 = scale8bit(red), scale8bit(green), scale8bit(blue)

# set rescaled fill pixels back to 0 for each array
red8[red == 0], green8[green == 0], blue8[blue == 0] = 0, 0, 0

rgb_stack = np.zeros((nrows,ncols,3),'uint8')
rgb_stack[...,0], rgb_stack[...,1], rgb_stack[...,2] = red8, green8, blue8

plt.rcParams['figure.figsize'] = [7,17]
plt.rcParams['figure.dpi'] = 100
plt.imshow(rgb_stack, vmin=0, vmax=.40)
plt.colorbar(fraction=0.04, pad=0.04)
plt.show()

## Apply a Histogram Stretch to for an Improved Visualization

In [ ]:
rgb_stack.shape

In [ ]:
# apply histogram equalization to each band
for i in range(rgb_stack.shape[2]):

    # band i
    b = rgb_stack[:,:,i]
    
    # histogram from flattened (1d) image
    b_histogram, bins = np.histogram(b.flatten(), 256)

    # cumulative distribution function
    b_cumdistfunc = b_histogram.cumsum()

    # normalize
    b_cumdistfunc = 255 * b_cumdistfunc / b_cumdistfunc[-1]

    # get new values by linear interpolation of cdf
    b_equalized = np.interp(b.flatten(), bins[:-1], b_cumdistfunc)
    
    # reshape to 2d and add back to rgb_stack
    rgb_stack[:,:,i] = b_equalized.reshape(b.shape)

plt.rcParams['figure.figsize'] = [7,17]
plt.rcParams['figure.dpi'] = 100
plt.imshow(rgb_stack, vmin=0, vmax=.40)
plt.colorbar(fraction=0.04, pad=0.04)
plt.show()

-----
-----

## Plot a Spectral Profile From a Point of Interest

In [ ]:
import matplotlib.pyplot as plt
from spectral.io import envi
import numpy as np
from pyproj import Proj
import os
from osgeo import gdal

### We'll plot the spectral profiles for a pixel from the Brackenburg Reserve.

Recall the Brackenburg Private Nature Reserve geographic location provided on their website

**[-33.978573 lat, 23.474745 lon]**

In [ ]:
# let's define the coordinates and data file from which we'll extract spectra and the lat/lon coordinates
data_file = 'data/ang20231110t081307_007_L2A_OE_main_27577724_RFL_ORT'
coords = [-33.978573, 23.474745]
coords

In [ ]:
data_file

## Geotransform

## Before we can extract pixel spectral information, we need to convert the lat/lon coordinate to an image coordinate (pixel, line) space

### - The point location is in a geographic coordinate system
### - We saw earlier, the image file is in a UTM zone 34S coordinate reference system

- 1. project the lat/lon point from geographic (units = decimal degrees) to the images projected (x,y) UTM coordinates (units = meters)
- 2. projected x,y (meter) coordinates need to be translated into a gridded offset of pixel, line values
- 3. extract pixel band values for the x,y offset a position 

#### 1. Project point location lat/lon to UTM (zone 34S): Recall, the AVIRIS-NG Scene is in UTM zone 34S (EPSG:32734)

In [ ]:
# step to translate coordinates
# EPSG:32734 - WGS-84 / UTM zone 34S

from pyproj import Proj
p = Proj("EPSG:32734", preserve_units=False)
x,y = p(23.474745, -33.978573)
print('x value in UTM meters:', x)
print('y value in UTM meters:', y)

In [ ]:
coords_UTM = [6237459.364997024, 728621.594343807]

# define aonther location for comparison
coords_UTM2 = [6239776.9, 730509.6]

In [ ]:
gdal_ds = gdal.Open(data_file)
proj_native = gdal_ds.GetProjection()
proj_native

#### 2. And now let's get and print AVIRIS-NG Scene information that we need in order to transform the UTM point x,y values to an image row/col location

In [ ]:
trans = gdal_ds.GetGeoTransform() # x_ul, x_px, x_rot, y_ul, y_rot, y_px
print('\nGetGeoTransform Return Values =', trans) 

#### GetGeotransform
**GT** is the geotransform acquired with img.GetGeoTransform()

-  GT(0): X origin, x_ul, **`730461.8657409984`**
-  GT(1): X resolution in the pixel space, x_px, **`5.0`**
-  GT(2). Represent the rotation of the pixel space from the geodetic space, x_rot, **`0.0`**
-  GT(3). Y origin, y_ul, **`6240060.191764411`**
-  GT(4). Represent the rotation of the pixel space from the geodetic space, y_rot, **`0.0`**
-  GT(5): -1 * Y resolution in the pixel space, y_px, **`-5.0`**

#### 3. x,y offset values are a position on the x,y grid of the pixel of interest

In [ ]:
x_px_offset = int(round((coords_UTM[1] - trans[0]) / trans[1]))
y_px_offset = int(round((coords_UTM[0] - trans[3]) / trans[5]))

print(x_px_offset, y_px_offset)

In [ ]:
x_px_offset2 = int(round((coords_UTM2[1] - trans[0]) / trans[1]))
y_px_offset2 = int(round((coords_UTM2[0] - trans[3]) / trans[5]))

print(x_px_offset2, y_px_offset2)

In [ ]:
ds = envi.open(os.path.splitext(data_file)[0] + '.hdr')
ds

In [ ]:
offset_size = 1
data = ds.open_memmap(interleave='bip')[y_px_offset - 1: y_px_offset + 1, x_px_offset - 1: x_px_offset + 1, :]
data

In [ ]:
offset_size = 1
data2 = ds.open_memmap(interleave='bip')[y_px_offset2 - 1: y_px_offset2 + 1, x_px_offset2 - 1: x_px_offset2 + 1, :]
data2

In [ ]:
wl = np.array([float(x) for x in ds.metadata['wavelength']])
wl2 = np.array([float(x) for x in ds.metadata['wavelength']])

plt.plot(wl, np.mean(data, axis=(0, 1)))
plt.rcParams['figure.figsize'] = [8, 8]
plt.xlabel('Wavelength', fontsize=20)
plt.ylabel('Reflectance', fontsize=20)
plt.show()

# plot the average spectrum

In [ ]:
# Define a list of wavelengths that are "bad" 

bblist = np.ones((425,))  # create a 1D array with values ones
# set tails and atmospheric window to zero
bblist[0:14] = 0        # tail
bblist[189:225] = 0     # atmospheric window
bblist[281:336] = 0     # atmospheric window
bblist[405:] = 0        # tail

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]

wl[bblist == 0] = np.nan 
wl2[bblist == 0] = np.nan
plt.plot(wl, np.mean(data, axis=(0, 1)), color = 'g')
plt.plot(wl2, np.mean(data2, axis=(0, 1)), color = 'r')
plt.xlabel('Wavelength', fontsize=20)
plt.ylabel('Reflectance', fontsize=20)
plt.show()

## Congrats! - You're on your way to exploring AVIRIS-NG data from the BioSCape Project